In [1]:
import time
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import sklearn.metrics as metrics

Apply the needed transformation to the data, as required by the Pytorch framework. <br/>
Then, upload the data into DataLoader objects, which will be given as inputs to the model.


In [2]:
# Source: 
# https://www.learnopencv.com/image-classification-using-transfer-learning-in-pytorch/

# Applying Transforms to the Data. These are needed for compatibility with the pytorch implementation of the model 
image_transforms = {
    'train': transforms.Compose([
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}


# Set train and valid directory paths
train_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/train'
valid_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/valid'
test_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/test'
 
# Batch size
bs = 64
 
# Number of classes
num_classes = 2
 
# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid']),
    'test': datasets.ImageFolder(root=test_directory, transform=image_transforms['test'])
}
 
# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])
 
# Create iterators for the Data loaded using DataLoader module
train_data = torch.utils.data.DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data = torch.utils.data.DataLoader(data['valid'], batch_size=bs, shuffle=True)
test_data = torch.utils.data.DataLoader(data['test'], batch_size=bs, shuffle=True)
 
# Print the train, validation and test set data sizes
print(f"Train size: {train_data_size} \nValidation size: {valid_data_size} \nTest size: {test_data_size}") 

Train size: 151293 
Validation size: 35876 
Test size: 35659


In [3]:
# Load pretrained AlexNet Model
alexnet = torchvision.models.alexnet(pretrained=True)

In [4]:
# Optional: load a pretrained DenseNet model.
densenet = torchvision.models.densenet161(pretrained=True)

In [5]:
# Select the model (alexnet/densenet)
model = densenet

Since the model is pretrained, the parameters can be "frozen"

In [ ]:
# Freeze the parameters for the pretrained part
# Source: https://pytorch.org/docs/master/notes/autograd.html
for param in model.parameters():
    param.requires_grad = False

Update the last layers of the model, in order to output only 2 classes.

In [6]:
# Note: both models must be loaded, for this to work
if model == densenet:
    
    # Update the classifier layer, in order to output the required number of classes specific to our problem
    # Note: see model.eval() for details of each layer
    model.classifier = nn.Linear(in_features=2208, out_features=num_classes, bias=True)

else:
    
    # Source: 
    # https://analyticsindiamag.com/implementing-alexnet-using-pytorch-as-a-transfer-learning-model-in-multi-class-classification/
    # Updating the second classifier(reduce the number of outputs, to prevent overfitting)
    model.classifier[4] = nn.Linear(4096,1024)

    # Updating the third and the last classifier that is the output layer of the network
    # Binary classification , thus only 2 output nodes
    model.classifier[6] = nn.Linear(1024, num_classes)

model_name = model.__class__.__name__
print(model_name)

DenseNet


Define loss function: Cross Entropy Loss

Note: Improvement option by adding weight class check

In [7]:
# Possible Improvement
# Source: 
# https://github.com/choosehappy/PytorchDigitalPathology/blob/master/visualization_densenet/train_densenet.ipynb

# "we have the ability to weight individual classes, in this case we'll do so based on their presense in the trainingset
# to avoid biasing any particular class"
# nclasses = dataset["train"].classsizes.shape[0]
# class_weight=dataset["train"].classsizes
# class_weight = torch.from_numpy(1-class_weight/class_weight.sum()).type('torch.FloatTensor').to(device)
# print(class_weight) #show final used weights, make sure that they're reasonable before continouing


# criterion = torch.nn.CrossEntropyLoss(weight = class_weight)
criterion = nn.CrossEntropyLoss()


Define optimizer: Adam

In [ ]:
# Source: 
# https://github.com/choosehappy/PytorchDigitalPathology/blob/master/visualization_densenet/train_densenet.ipynb

# adam is going to be the most robust, though perhaps not the best performing, typically a good place to start
optimizer = optim.Adam(model.parameters()) 

If GPU is available, use it. Otherwise, use the CPU.

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"You are running on the following device: {device}")

You are running on the following device: cpu


**Print model and optimizer parameters before training**

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
history = []

# Initialize the variable where accuracy and loss are stored (with max/min values)
history.append([1.0, 1.0, 0.0, 0.0])
best_loss_on_val = np.Infinity

epochs = 20
for epoch in range(epochs):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch, epochs))
    
    # Set gradient calculation to ON. Needed during training.
    torch.set_grad_enabled(True)
        
    # Set to training mode
    model.train()
    
    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0
     
    valid_loss = 0.0
    valid_acc = 0.0
 
    # Iterate through all batches of training data
    for i, (inputs, labels) in enumerate(train_data):
 
        inputs = inputs.to(device)
        labels = labels.to(device)
         
        # Clean existing gradients
        optimizer.zero_grad()
         
        # Forward pass - compute outputs on input data using the model
        outputs = model(inputs)
                 
        # Compute loss
        loss = criterion(outputs, labels)
         
        # Backpropagate the gradients
        loss.backward()
         
        # Update the parameters
        optimizer.step()
         
        # Compute the total loss for the batch and add it to train_loss
        train_loss += loss.item() * inputs.size(0)
         
        # Compute the accuracy
        ret, predictions = torch.max(outputs.data, 1)

        correct_counts = predictions.eq(labels.data.view_as(predictions))
         
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
         
        # Compute total accuracy in the whole batch and add to train_acc
        train_acc += acc.item() * inputs.size(0)

         
        print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

        # Break "train_data batch" for loop
        # break
    
        

    # Validation is carried out in each epoch immediately after the training loop
    # Validation - No gradient calculation is needed.
    with torch.no_grad():

        # Set to evaluation mode
        model.eval()

        # Validation loop
        # Iterate through all batches of validation data
        for j, (inputs, labels) in enumerate(valid_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            valid_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            valid_acc += acc.item() * inputs.size(0)


            print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

            # Break "valid_data batch" for loop
            # break

    # Find average training loss and training accuracy
    avg_train_loss = train_loss/train_data_size
    avg_train_acc = train_acc/float(train_data_size)

    # Find average validation loss and validation accuracy
    avg_valid_loss = valid_loss/valid_data_size
    avg_valid_acc = valid_acc/float(valid_data_size)

    history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

    epoch_end = time.time()

    print(f"Model: {model_name} \n")
    print("Epoch : {:03d} \nTraining: Loss: {:.4f}, Accuracy: {:.4f}%, \nValidation : Loss : {:.4f}, Accuracy: {:.4f}%, \nTime (train+val): {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

    
    # Source: https://github.com/choosehappy/PytorchDigitalPathology/blob/master/classification_lymphoma_densenet/train_densenet.ipynb
    # If current loss is the best we've seen, save model state with all variables
    # necessary for recreation
    if avg_valid_loss < best_loss_on_val:
        best_loss_on_val = avg_valid_loss
        print("  **")
        state = {'epoch': epoch + 1,
         'model_dict': model.state_dict(),
         'optim_dict': optimizer.state_dict(),
         'loss': criterion,
         'best_loss_on_val': best_loss_on_val}

        torch.save(state, f"outputs/{model_name}_best_model.pth")
        print(f"Saved model {model_name} with loss {avg_valid_loss}")
    else:
        print("")
    
    # Stop "epoch" for
    # break

**Take a look at the model structure**

In [15]:
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

**Plot the accuracy and loss**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.style.use('ggplot')

# accuracy plots
plt.figure(figsize=(10, 7))
plt.plot(range(len(history)), [x[3] for x in history] , color='green', label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('outputs/initial_validation_accuracy.png')
plt.show()
 
# loss plots
plt.figure(figsize=(10, 7))
plt.plot(range(len(history)), [x[1] for x in history], color='orange', label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('outputs/initial_training_loss.png')
plt.show()

**Print model and optimizer parameters after training**

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

**Save the model**

In [ ]:
# Source:
# https://debuggercafe.com/effective-model-saving-and-resuming-training-in-pytorch/
# save model checkpoint
torch.save({
            'epoch': epochs,
            'model_dict': model.state_dict(),
            'optim_dict': optimizer.state_dict(),
            'loss': criterion,
            }, f'outputs/{model_name}_trained.pth')

**Initialize the model before loading the previously saved one**

In [10]:
if model == alexnet:

    # Initialize the model
    model_loaded = torchvision.models.alexnet(pretrained=True)
    
    # Updating the second classifier
    model_loaded.classifier[4] = nn.Linear(4096,1024)

    # Binary classification , thus only 2 output nodes
    model_loaded.classifier[6] = nn.Linear(1024, num_classes)
    
if model == densenet:
    
    # Optional: load a pretrained DenseNet model.
    model_loaded = torchvision.models.densenet161(pretrained=True)
    
    # Update the classifier layer, in order to output the required number of classes specific to our problem
    # Note: see model.eval() for details of each layer
    model_loaded.classifier = nn.Linear(in_features=2208, out_features=num_classes, bias=True)
    
    
# Initialize optimizer  before loading optimizer state_dict
optimizer_loaded = optim.Adam(model_loaded.parameters()) 

    

**Load the saved model**

In [11]:
# load the model checkpoint
# checkpoint = torch.load(f'outputs/{model_name}_trained.pth')
checkpoint = torch.load(f'outputs/{model_name}_best_model.pth')

# load model weights state_dict
model_loaded.load_state_dict(checkpoint['model_dict'])
print('Previously trained model weights state_dict loaded...')

# load trained optimizer state_dict
# optimizer_loaded.load_state_dict(checkpoint['optim_dict'])
# print('Previously trained optimizer state_dict loaded...')

epochs = checkpoint['epoch']

# load the criterion
# criterion_loaded = checkpoint['loss']
# print('Trained model loss function loaded...')

print(f"Previously trained for {epochs} number of epochs...")


Previously trained model weights state_dict loaded...
Previously trained for 2 number of epochs...


**Print loaded model and optimizer parameters**

In [12]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model_loaded.state_dict():
    print(param_tensor, "\t", model_loaded.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer_loaded.state_dict():
    print(var_name, "\t", optimizer_loaded.state_dict()[var_name])

Model's state_dict:
features.conv0.weight 	 torch.Size([96, 3, 7, 7])
features.norm0.weight 	 torch.Size([96])
features.norm0.bias 	 torch.Size([96])
features.norm0.running_mean 	 torch.Size([96])
features.norm0.running_var 	 torch.Size([96])
features.norm0.num_batches_tracked 	 torch.Size([])
features.denseblock1.denselayer1.norm1.weight 	 torch.Size([96])
features.denseblock1.denselayer1.norm1.bias 	 torch.Size([96])
features.denseblock1.denselayer1.norm1.running_mean 	 torch.Size([96])
features.denseblock1.denselayer1.norm1.running_var 	 torch.Size([96])
features.denseblock1.denselayer1.norm1.num_batches_tracked 	 torch.Size([])
features.denseblock1.denselayer1.conv1.weight 	 torch.Size([192, 96, 1, 1])
features.denseblock1.denselayer1.norm2.weight 	 torch.Size([192])
features.denseblock1.denselayer1.norm2.bias 	 torch.Size([192])
features.denseblock1.denselayer1.norm2.running_mean 	 torch.Size([192])
features.denseblock1.denselayer1.norm2.running_var 	 torch.Size([192])
features.de

features.denseblock2.denselayer10.norm1.running_mean 	 torch.Size([624])
features.denseblock2.denselayer10.norm1.running_var 	 torch.Size([624])
features.denseblock2.denselayer10.norm1.num_batches_tracked 	 torch.Size([])
features.denseblock2.denselayer10.conv1.weight 	 torch.Size([192, 624, 1, 1])
features.denseblock2.denselayer10.norm2.weight 	 torch.Size([192])
features.denseblock2.denselayer10.norm2.bias 	 torch.Size([192])
features.denseblock2.denselayer10.norm2.running_mean 	 torch.Size([192])
features.denseblock2.denselayer10.norm2.running_var 	 torch.Size([192])
features.denseblock2.denselayer10.norm2.num_batches_tracked 	 torch.Size([])
features.denseblock2.denselayer10.conv2.weight 	 torch.Size([48, 192, 3, 3])
features.denseblock2.denselayer11.norm1.weight 	 torch.Size([672])
features.denseblock2.denselayer11.norm1.bias 	 torch.Size([672])
features.denseblock2.denselayer11.norm1.running_mean 	 torch.Size([672])
features.denseblock2.denselayer11.norm1.running_var 	 torch.Size

features.denseblock3.denselayer12.norm2.bias 	 torch.Size([192])
features.denseblock3.denselayer12.norm2.running_mean 	 torch.Size([192])
features.denseblock3.denselayer12.norm2.running_var 	 torch.Size([192])
features.denseblock3.denselayer12.norm2.num_batches_tracked 	 torch.Size([])
features.denseblock3.denselayer12.conv2.weight 	 torch.Size([48, 192, 3, 3])
features.denseblock3.denselayer13.norm1.weight 	 torch.Size([960])
features.denseblock3.denselayer13.norm1.bias 	 torch.Size([960])
features.denseblock3.denselayer13.norm1.running_mean 	 torch.Size([960])
features.denseblock3.denselayer13.norm1.running_var 	 torch.Size([960])
features.denseblock3.denselayer13.norm1.num_batches_tracked 	 torch.Size([])
features.denseblock3.denselayer13.conv1.weight 	 torch.Size([192, 960, 1, 1])
features.denseblock3.denselayer13.norm2.weight 	 torch.Size([192])
features.denseblock3.denselayer13.norm2.bias 	 torch.Size([192])
features.denseblock3.denselayer13.norm2.running_mean 	 torch.Size([192])

features.denseblock3.denselayer28.norm1.running_mean 	 torch.Size([1680])
features.denseblock3.denselayer28.norm1.running_var 	 torch.Size([1680])
features.denseblock3.denselayer28.norm1.num_batches_tracked 	 torch.Size([])
features.denseblock3.denselayer28.conv1.weight 	 torch.Size([192, 1680, 1, 1])
features.denseblock3.denselayer28.norm2.weight 	 torch.Size([192])
features.denseblock3.denselayer28.norm2.bias 	 torch.Size([192])
features.denseblock3.denselayer28.norm2.running_mean 	 torch.Size([192])
features.denseblock3.denselayer28.norm2.running_var 	 torch.Size([192])
features.denseblock3.denselayer28.norm2.num_batches_tracked 	 torch.Size([])
features.denseblock3.denselayer28.conv2.weight 	 torch.Size([48, 192, 3, 3])
features.denseblock3.denselayer29.norm1.weight 	 torch.Size([1728])
features.denseblock3.denselayer29.norm1.bias 	 torch.Size([1728])
features.denseblock3.denselayer29.norm1.running_mean 	 torch.Size([1728])
features.denseblock3.denselayer29.norm1.running_var 	 torc

features.denseblock4.denselayer8.norm1.num_batches_tracked 	 torch.Size([])
features.denseblock4.denselayer8.conv1.weight 	 torch.Size([192, 1392, 1, 1])
features.denseblock4.denselayer8.norm2.weight 	 torch.Size([192])
features.denseblock4.denselayer8.norm2.bias 	 torch.Size([192])
features.denseblock4.denselayer8.norm2.running_mean 	 torch.Size([192])
features.denseblock4.denselayer8.norm2.running_var 	 torch.Size([192])
features.denseblock4.denselayer8.norm2.num_batches_tracked 	 torch.Size([])
features.denseblock4.denselayer8.conv2.weight 	 torch.Size([48, 192, 3, 3])
features.denseblock4.denselayer9.norm1.weight 	 torch.Size([1440])
features.denseblock4.denselayer9.norm1.bias 	 torch.Size([1440])
features.denseblock4.denselayer9.norm1.running_mean 	 torch.Size([1440])
features.denseblock4.denselayer9.norm1.running_var 	 torch.Size([1440])
features.denseblock4.denselayer9.norm1.num_batches_tracked 	 torch.Size([])
features.denseblock4.denselayer9.conv1.weight 	 torch.Size([192, 144

features.denseblock4.denselayer24.conv2.weight 	 torch.Size([48, 192, 3, 3])
features.norm5.weight 	 torch.Size([2208])
features.norm5.bias 	 torch.Size([2208])
features.norm5.running_mean 	 torch.Size([2208])
features.norm5.running_var 	 torch.Size([2208])
features.norm5.num_batches_tracked 	 torch.Size([])
classifier.weight 	 torch.Size([2, 2208])
classifier.bias 	 torch.Size([2])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,

**Initialize parameters for second phase of training (optional)**

In [ ]:
# learning parameters
batch_size = 64
new_epochs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# train for more epochs
epochs = new_epochs
print(f"Train for {epochs} more epochs...")

In [ ]:
history_loaded = []

for epoch in range(epochs):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch, epochs))
     
    # Set gradient calculation to ON. Needed during training.
    torch.set_grad_enabled(True)
        
    # Set to training mode
    model_loaded.train()
     
    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0
     
    valid_loss = 0.0
    valid_acc = 0.0
 
    # Iterate through all batches of training data
    for i, (inputs, labels) in enumerate(train_data):
 
        inputs = inputs.to(device)
        labels = labels.to(device)
         
        # Clean existing gradients
        optimizer_loaded.zero_grad()
         
        # Forward pass - compute outputs on input data using the model
        outputs = model_loaded(inputs)
        
        # Compute loss
        loss = criterion_loaded(outputs, labels)
         
        # Backpropagate the gradients
        loss.backward()
         
        # Update the parameters
        optimizer_loaded.step()
         
        # Compute the total loss for the batch and add it to train_loss
        train_loss += loss.item() * inputs.size(0)
         
        # Compute the accuracy
        ret, predictions = torch.max(outputs.data, 1)

        correct_counts = predictions.eq(labels.data.view_as(predictions))
         
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
         
        # Compute total accuracy in the whole batch and add to train_acc
        train_acc += acc.item() * inputs.size(0)

         
        print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))
        

    # Validation is carried out in each epoch immediately after the training loop
    # Validation - No gradient calculation is needed
    with torch.no_grad():

        # Set to evaluation mode
        model_loaded.eval()

        # Validation loop
        # Iterate through all batches of validation data
        for j, (inputs, labels) in enumerate(valid_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model_loaded(inputs)

            # Compute loss
            loss = criterion_loaded(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            valid_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            valid_acc += acc.item() * inputs.size(0)


            print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

    # Find average training loss and training accuracy
    avg_train_loss = train_loss/train_data_size
    avg_train_acc = train_acc/float(train_data_size)

    # Find average training loss and training accuracy
    avg_valid_loss = valid_loss/valid_data_size
    avg_valid_acc = valid_acc/float(valid_data_size)

    history_loaded.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

    epoch_end = time.time()

    print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, \n Time (train+val): {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))


In [13]:
# Check if the current model was loaded, or it was just trained. 
# Needed for compatibility with testing stage.
if model_loaded: 
    model = model_loaded
    criterion = nn.CrossEntropyLoss()

**Evaluate model based on test dataset**

In [14]:
test_start = time.time()

# Loss and Accuracy within the epoch
test_loss = 0.0
test_acc = 0.0

# Initialize an empty tensor. This will store all the predictions and will be used for metrics.
tot_predictions = torch.Tensor()
# Initialize an empty tensor. This will store all the ground truth labels and will be used for metrics.
tot_labels = torch.Tensor()

# Testing - No gradient calculation is needed
with torch.no_grad():

    # Set to evaluation mode
    model.eval()

    # Validation loop
    # Iterate through all batches of test data
    for j, (inputs, labels) in enumerate(test_data):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Add values to ground truth
        tot_labels = torch.cat((tot_labels, labels))

        # Forward pass - compute outputs on input data using the model
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Compute the total loss for the batch and add it to test_loss
        test_loss += loss.item() * inputs.size(0)

        # Calculate validation accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
        tot_predictions = torch.cat((tot_predictions, predictions))

        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))

        # Compute total accuracy in the whole batch and add to test_acc
        test_acc += acc.item() * inputs.size(0)

#             print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))


# Find average testing loss and accuracy
avg_test_loss = test_loss/test_data_size
avg_test_acc = test_acc/float(test_data_size)

accuracy = metrics.accuracy_score(tot_labels.to(device), tot_predictions.to(device))

# The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
precision = metrics.precision_score(tot_labels.to(device), tot_predictions.to(device), average='binary')

# The recall is intuitively the ability of the classifier to find all the positive samples.
recall = metrics.recall_score(tot_labels.to(device), tot_predictions.to(device), pos_label=1, average='binary')

# The F1 score can be interpreted as a weighted average of the precision and recall
f1 = metrics.f1_score(tot_labels.to(device), tot_predictions.to(device), average='binary')

# history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

test_end = time.time()
print(f"accuracy = {accuracy}")
print(f"precision = {precision}")
print(f"recall = {recall}")
print(f"f1 = {f1}")
print("Test \nLoss : {:.4f}, Accuracy: {:.4f}%, \nTime : {:.4f}s".format(avg_test_loss, avg_test_acc*100, test_end-test_start))

accuracy = 0.8409658150817465
precision = 0.8967206558688262
recall = 0.6591460277798192
f1 = 0.759794993434707
Test 
Loss : 0.3794, Accuracy: 84.0966%, 
Time : 5728.6465s
